In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#### Calibración de hiperparámetros y regularización

data = sns.load_dataset('titanic')
data.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [ ]:
VARS = ['pclass','sex','age','sibsp','parch','fare']
TARGET = ['survived']
x = data[VARS]
y = data[TARGET]

In [ ]:
### Train test split

from sklearn.model_selection import train_test_split


x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state = 123)

In [ ]:
### Dataset de validación

x_train_d, x_val, y_train_d, y_val = train_test_split(x_train, y_train, test_size = 0.2, random_state = 123)

In [ ]:
### Variables categóricas

from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
x_train_cat = pd.DataFrame(enc.fit_transform(x_train_d[['sex']]).toarray(), index = x_train_d.index, columns = enc.categories_[0])
x_val_cat = pd.DataFrame(enc.transform(x_val[['sex']]).toarray(), index = x_val.index, columns = enc.categories_[0])
x_test_cat = pd.DataFrame(enc.transform(x_test[['sex']]).toarray(), index = x_test.index, columns = enc.categories_[0])
x_train_total = pd.DataFrame(enc.transform(x_train[['sex']]).toarray(), index = x_train.index, columns = enc.categories_[0])

In [ ]:
### Preprocesamiento de datos

from sklearn.impute import KNNImputer

imputer = KNNImputer()

x_train_no_missing = pd.DataFrame(imputer.fit_transform(x_train_d[['pclass','age','sibsp','parch','fare']]), index = x_train_d.index, columns = ['pclass','age','sibsp','parch','fare'])
x_val_no_missing = pd.DataFrame(imputer.transform(x_val[['pclass','age','sibsp','parch','fare']]), index = x_val.index, columns = ['pclass','age','sibsp','parch','fare'])
x_test_no_missing = pd.DataFrame(imputer.transform(x_test[['pclass','age','sibsp','parch','fare']]), index = x_test.index, columns = ['pclass','age','sibsp','parch','fare'])
x_train_no_missing_final = pd.DataFrame(imputer.transform(x_train[['pclass','age','sibsp','parch','fare']]), index = x_train.index, columns = ['pclass','age','sibsp','parch','fare'])

In [ ]:
x_train_final = pd.concat([x_train_cat, x_train_no_missing], axis = 1)
x_val_final = pd.concat([x_val_cat, x_val_no_missing], axis = 1)
x_test_final = pd.concat([x_test_cat, x_test_no_missing], axis = 1)
x_train_total_final = pd.concat([x_train_total, x_train_no_missing_final], axis = 1)

In [ ]:
### Escalamiento de datos

from sklearn.preprocessing import StandardScaler

esc = StandardScaler()
x_train_final_esc = pd.DataFrame(esc.fit_transform(x_train_final), index = x_train_final.index, columns = x_train_final.columns)
x_val_final_esc = pd.DataFrame(esc.transform(x_val_final), index = x_val_final.index, columns = x_val_final.columns)
x_test_final_esc = pd.DataFrame(esc.transform(x_test_final), index = x_test_final.index, columns = x_test_final.columns)
x_train_final_total_esc = pd.DataFrame(esc.transform(x_train_total_final), index = x_train_total_final.index, columns = x_train_total_final.columns)

In [ ]:
### Modelamiento

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

modelo_uno = LogisticRegression(penalty = 'l2', solver = 'saga', C = 0.04)
modelo_dos = LogisticRegression(penalty = 'l1', solver = 'saga', C = 0.02)
modelo_tres = LogisticRegression(penalty = 'elasticnet', solver = 'saga', C = 0.04, l1_ratio = 0.8)
modelo_arbol = DecisionTreeClassifier(max_depth = 19)


def training(modelo, x_train, y_train, x_val, y_val):
  modelo.fit(x_train, y_train)
  preds = modelo.predict(x_val)
  score = accuracy_score(preds, y_val)
  print(score)
  return modelo, preds, score

training(modelo_uno, x_train_final_esc, y_train_d, x_val_final_esc, y_val)
training(modelo_dos, x_train_final_esc, y_train_d, x_val_final_esc, y_val)
training(modelo_tres, x_train_final_esc, y_train_d, x_val_final_esc, y_val)
training(modelo_arbol, x_train_final_esc, y_train_d, x_val_final_esc, y_val)

0.8181818181818182
0.8041958041958042
0.8041958041958042
0.7902097902097902


(DecisionTreeClassifier(max_depth=19),
 array([0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1,
        0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0,
        0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0,
        1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
        0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0,
        1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0]),
 0.7902097902097902)

In [ ]:
### GridSearchCV
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
modelo_uno = LogisticRegression(penalty = 'l2', solver = 'saga', C = 0.04)
parameters = {'penalty':('l1', 'l2'), 'C': np.arange(0.01,1,0.01)}

grid_search = GridSearchCV(modelo_uno, parameters)
grid_search.fit(x_train_final_total_esc, y_train)





GridSearchCV(estimator=LogisticRegression(C=0.04, solver='saga'),
             param_grid={'C': array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66,
       0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77,
       0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88,
       0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99]),
                         'penalty': ('l1', 'l2')})

In [ ]:
modelo_final = LogisticRegression(**grid_search.best_params_)
modelo_final.fit(x_train_final_esc, y_train_d)
preds = modelo_final.predict(x_test_final_esc)
accuracy_score(preds, y_test)

0.7932960893854749

In [ ]:
### RandomSearchCV
modelo_uno = LogisticRegression(penalty = 'l2', solver = 'saga', C = 0.04)
parameters = {'penalty':('l1', 'l2'), 'C': np.arange(0.01,1,0.01)}

random_search = RandomizedSearchCV(modelo_uno, parameters)
random_search.fit(x_train_final_total_esc, y_train)


RandomizedSearchCV(estimator=LogisticRegression(C=0.04, solver='saga'),
                   param_distributions={'C': array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66,
       0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77,
       0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88,
       0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99]),
                                        'penalty': ('l1', 'l2')})

In [ ]:
random_search.best_params_

{'penalty': 'l2', 'C': 0.41000000000000003}

In [ ]:
modelo_final = LogisticRegression(**random_search.best_params_)
modelo_final.fit(x_train_final_esc, y_train_d)
preds = modelo_final.predict(x_test_final_esc)
accuracy_score(preds, y_test)

0.8044692737430168